In [ ]:
import logging
import re
import time
from datetime import datetime
from typing import Dict, List, Optional, Set
from urllib.parse import quote_plus
import os
import shutil
import tempfile
import pandas as pd

# Dependencias de Selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException, NoSuchElementException, StaleElementReferenceException,
    WebDriverException, SessionNotCreatedException,
)
from webdriver_manager.chrome import ChromeDriverManager


# ------------------- Configuración de Logging -------------------
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
# ------------------------------------------------------------------------------


# --- BLOQUE BASE.PY (Se mantiene la estructura estable) ---

class BaseScraper:
    """Configuración común para scrapers basados en Selenium (uso local con webdriver-manager)."""

    def __init__(self, data_dir: str = "data"):
        self.data_dir = data_dir
        os.makedirs(self.data_dir, exist_ok=True)
        self._temp_dir = None
        
        headless_mode = False 

        def build_options() -> webdriver.ChromeOptions:
            opts = webdriver.ChromeOptions()
            
            opts.add_argument("--lang=es-ES,es;q=0.9")
            opts.add_argument(
                "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
            )
            opts.add_argument("--no-sandbox")
            opts.add_argument("--disable-dev-shm-usage")
            opts.add_argument("--disable-gpu")
            opts.add_argument("--window-size=1366,900")
            opts.add_experimental_option("excludeSwitches", ["enable-automation"])
            opts.add_experimental_option("useAutomationExtension", False)
            opts.add_argument("--disable-blink-features=AutomationControlled")
            
            return opts

        options = build_options()
        
        try:
            service = Service(ChromeDriverManager().install())
            self.driver = webdriver.Chrome(service=service, options=options)

        except Exception as e:
            logging.error(f"Error al iniciar Chrome: {e}")
            raise e

        try:
            self.driver.execute_cdp_cmd(
                "Page.addScriptToEvaluateOnNewDocument",
                {"source": """
                    Object.defineProperty(navigator,'webdriver',{get:()=>undefined});
                    window.chrome = window.chrome || {};
                    window.chrome.app = {isInstalled: false};
                    Object.defineProperty(navigator, 'plugins', {get: () => [1,2,3,4]});
                    Object.defineProperty(navigator, 'mimeTypes', {get: () => [1,2,3,4]});
                    Object.defineProperty(navigator, 'languages', {get: () => ['es-ES','es']});
                """}
            )
        except Exception: pass

    def __enter__(self): return self
    def __exit__(self, exc_type, exc, tb): self.close()
    
    def close(self):
        if getattr(self, "driver", None):
            try: self.driver.quit()
            except Exception: pass
            self.driver = None
        if getattr(self, "_temp_dir", None):
            try: self._temp_dir.cleanup()
            except Exception: pass
            self._temp_dir = None

# --- FIN DE BASE.PY ---

# --- COMIENZO DE UTILIDADES Y CLASE ALIBABA SCRAPER ---

_RANGE_SPLIT_PATTERN = re.compile(r"(?<=\d)\s*[-–—]\s*(?=\d)")
_PRODUCT_ID_RE = re.compile(r"productId=(\d+)")
_P4P_ID_RE = re.compile(r"p4pid=([a-f0-9]+)") 
_RLT_RANK_RE = re.compile(r"rlt_rank:(\d+)") 
_IS_P4P_RE = re.compile(r"is_p4p=(true|false)")
_COUNTRY_CODE_ATTR_RE = re.compile(r"areaContent=(\w{2})@@")

# Funciones de limpieza y parsing (se mantienen)
def limpiar_precio(texto: Optional[str]) -> Optional[float]:
    def _normalizar(texto_unitario: str) -> Optional[float]:
        try:
            return float(re.sub(r"[^\d.]", "", texto_unitario))
        except ValueError:
            pass
        cleaned = re.sub(r"[^0-9.,]", "", texto_unitario)
        if not cleaned: return None
        if "," in cleaned and cleaned.rfind(",") > cleaned.rfind("."):
            return float(cleaned.replace('.', '').replace(',', '.'))
        elif "." in cleaned and cleaned.rfind(".") > cleaned.rfind(","):
            return float(cleaned.replace(',', ''))
        if re.search(r"[\d][.,][\d]{1,2}$", cleaned):
             if cleaned.endswith(','):
                return float(cleaned.replace('.', '').replace(',', '.'))
             else:
                return float(cleaned.replace(',', ''))
        number_str = re.sub(r"[^0-9]", "", cleaned)
        if not number_str: return None
        try: return float(number_str)
        except ValueError: return None
    
    if not texto: return None
    texto = texto.strip(); 
    if not texto: return None
    if re.search(r"[-–—]", texto):
        partes = re.split(r"[-–—]", texto)
        for p in partes:
            if (v := _normalizar(p.strip())) is not None: return v
        if partes: texto = partes[0]
    return _normalizar(texto)

def limpiar_cantidad(texto: Optional[str]) -> int:
    if texto is None: return 0
    t = texto.strip().lower().replace("+", "")
    if not t: return 0
    mult = 1
    if re.search(r"k\b", t): mult = 1000; t = re.sub(r"k\b", "", t)
    if "mil" in t: mult = max(mult, 1000); t = t.replace("mil", "")
    n = limpiar_precio(t) or 0.0
    if n > 1000 and "." in t and t.count('.') == 1 and t.split('.')[-1].isdigit():
        n = float(t.replace('.', '')) / mult 
    return int(round(n * mult))

_currency_re = re.compile(r"(US\$|S/|[$€£¥]|USD)")
_rating_re = re.compile(r"([\d.]+)\s*/\s*5(?:\.0)?\s*\((\d+)\)")
_years_re  = re.compile(r"(\d+)\s*(?:años|years?)", re.I)

def detectar_moneda(texto: str) -> Optional[str]:
    if not texto: return None
    m = _currency_re.search(texto)
    return m.group(1) if m else None

def parse_rating(texto: str) -> tuple[Optional[float], Optional[int]]:
    if not texto: return (None, None)
    m = _rating_re.search(texto)
    if not m: return (None, None)
    try: return float(m.group(1)), int(m.group(2))
    except: return (None, None)

def parse_years_country(node) -> tuple[Optional[int], Optional[str]]:
    text = ""; country = None
    years = None
    
    try:
        aplus_mod = node.get_attribute("data-aplus-auto-card-mod") or ""
        m_country = _COUNTRY_CODE_ATTR_RE.search(aplus_mod)
        if m_country:
            country = m_country.group(1).strip()
    except Exception:
        pass
    
    try: text = (node.text or "").strip()
    except Exception: pass
    m = _years_re.search(text)
    if m:
        try: years = int(m.group(1))
        except: years = None
        
    if not country:
        try:
            img = node.find_element(By.CSS_SELECTOR, "img[alt]")
            country = (img.get_attribute("alt") or "").strip() or None
        except Exception:
            pass
            
    return years, country

def parse_moq(texto: str) -> tuple[Optional[int], Optional[str]]:
    if not texto: return (None, None)
    m = re.search(r"(\d[\d.,]*)", texto)
    if not m: return (None, texto.strip())
    try: val = limpiar_cantidad(m.group(1))
    except: val = None
    return val, texto.strip()


class AlibabaScraper(BaseScraper):
    """Scraper Alibaba con robustez y enfoque en las 16 columnas relevantes."""

    CARD_CONTAINERS: List[str] = [
        "div.fy26-product-card-wrapper", "div.__fy26-product-card-wrapper",
        "div.searchx-product-card", "div.card-info.gallery-card-layout-info",
    ]

    A_CARD: List[str] = ["h2.searchx-product-e-title a", "a.searchx-product-link-wrapper", "a"]
    TITLE: List[str] = ["h2.searchx-product-e-title span", "h2.searchx-product-e-title a", "h2.search-card-e-title"]
    PRICE: List[str] = [
        "div.searchx-product-price-price-main", "div.searchx-product-price", 
        ".price--two-line", "div[data-aplus-auto-card-mod*='area=price'] div", "div.price"
    ]
    PRICE_ORIGINAL: List[str] = ["del", "s", ".price-origin"]
    MOQ_CONTAINER: List[str] = ["div.searchx-moq"]
    SOLD_COUNT: List[str] = ["div.searchx-sold-order"]
    SUPPLIER_NAME: List[str] = ["a.searchx-product-e-company", "a.search-card-e-company"]
    SUPPLIER_YEAR_COUNTRY: List[str] = ["a.searchx-product-e-supplier__year"]
    VERIFIED_BADGE: List[str] = [".verified-supplier-icon__wrapper", "img.searchx-verified-icon"]
    RATING: List[str] = ["span.searchx-product-e-review"]
    AD_BADGE: List[str] = [".searchx-card-e-ad", "div[data-role='ad-area']"]
    SELLING_POINTS: List[str] = [".searchx-selling-point-text"]

    @classmethod
    def _resolve_text(cls, node) -> Optional[str]:
        if node is None: return None
        get_attribute = getattr(node, "get_attribute", None)
        if callable(get_attribute):
            content = get_attribute("textContent")
            if content: return content.strip()
            return (getattr(node, "text", "") or "").strip() or None
        return node.get_text(" ", strip=True) or None

    @classmethod
    def _resolve_price_text(cls, node, data_attribute: Optional[str] = None) -> Optional[str]:
        if node is None: return None
        get_attribute = getattr(node, "get_attribute", None)
        if callable(get_attribute):
            if data_attribute:
                v = get_attribute(data_attribute)
                if v: return v.strip()
            content = get_attribute("textContent")
            if content: return content.strip()
            return (get_attribute("innerText") or "").strip() or None
        return node.get_text(" ", strip=True) or None
    
    def _human_scroll_until_growth(self, max_scrolls: int = 16, pause: float = 1.0):
        logging.info("Iniciando scroll gradual...")
        last_height = 0
        scroll_count = 0
        while scroll_count < max_scrolls:
            try:
                self.driver.execute_script("window.scrollBy(0, 700);")
                time.sleep(pause)
                
                new_height = self.driver.execute_script("return document.body.scrollHeight")
                
                if new_height == last_height:
                    self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    time.sleep(pause)
                    final_height = self.driver.execute_script("return document.body.scrollHeight")
                    
                    if final_height <= new_height:
                        logging.info("Scroll gradual finalizado: No se detecta más contenido.")
                        break
                    last_height = final_height
                else:
                    last_height = new_height

                scroll_count += 1
            except Exception as e:
                logging.warning(f"Error durante el scroll: {e}")
                break
    
    def _first_match(self, root, selectors: List[str]):
        for css in selectors:
            try:
                elements = root.find_elements(By.CSS_SELECTOR, css)
                if elements: return elements[0]
            except Exception: continue
        return None

    def _find_all_any(self, selectors: List[str], timeout: int = 10) -> List:
        for css in selectors:
            try:
                WebDriverWait(self.driver, timeout).until(
                    EC.visibility_of_any_elements_located((By.CSS_SELECTOR, css)) 
                )
                els = self.driver.find_elements(By.CSS_SELECTOR, css)
                if els: return els
            except TimeoutException: continue
        return []
    
    @staticmethod
    def _abs_link(href: str) -> str:
        if not href: return ""
        if href.startswith("//"): return "https:" + href
        if href.startswith("/"): return "https://www.alibaba.com" + href
        if not href.startswith("http"): return "https://www.alibaba.com/" + href
        return href
        
    def _accept_banners(self, timeout: int = 5):
        """Intenta cerrar banners de cookies o pop-ups."""
        candidates = [
            (By.XPATH, "//button[contains(., 'Aceptar') or contains(., 'Accept')]"),
            (By.XPATH, "//button[contains(., 'Allow all')]"),
            (By.CSS_SELECTOR, "[role='button'][aria-label*='accept' i]"),
        ]
        for by, sel in candidates:
            try:
                btn = WebDriverWait(self.driver, timeout).until(EC.element_to_be_clickable((by, sel)))
                btn.click()
                time.sleep(0.3)
            except Exception:
                pass

    def _extract_card(self, card) -> Optional[Dict]:
        data = {}
        # logging.info("-" * 40) # Desactivamos el log de inicio para no sobrecargar la consola

        try:
            # --- Lectura Inmediata (Estrategia de Tolerancia) ---
            aplus_data_raw = card.get_attribute("data-aplus-auto-offer")
            data["product_id"] = card.get_attribute("data-ctrdot")
            
            aplus_data = aplus_data_raw if aplus_data_raw else ""
            
            # --- FASE 1: Identificación y Links (NO SE PUEDE FALLAR) ---
            a = self._first_match(card, self.A_CARD) or card
            data["link"] = self._abs_link((a.get_attribute("href") or "").strip())
            data["titulo"] = self._resolve_text(self._first_match(card, self.TITLE)) or "Sin título"
            
            # ID de Producto (CRÍTICO)
            if not data.get("product_id"): m = _PRODUCT_ID_RE.search(aplus_data); data["product_id"] = m.group(1) if m else None
            
            # logging.info(f"[DIAGNÓSTICO] Product ID: {data['product_id']}") # Desactivamos logs internos
            
            if not data["link"] and data["titulo"] == "Sin título": return None

            # PRECIO (Extracción con máxima robustez)
            price_text = None
            for price_sel in self.PRICE:
                price_el = self._first_match(card, [price_sel])
                if price_el:
                    price_text = self._resolve_price_text(price_el, "data-price")
                    if price_text: break
            
            if not price_text:
                 price_area_el = self._first_match(card, [".searchx-product-price-price-main"])
                 if price_area_el:
                     price_text = self._resolve_text(price_area_el)
            
            data["precio"] = limpiar_precio(price_text)
            data["moneda"] = detectar_moneda(price_text or "") if price_text else None
            
            # Transacciones (A menudo son None si el JS falla, pero lo intentamos)
            moq_el = self._first_match(card, self.MOQ_CONTAINER)
            data["moq"], data["moq_texto"] = parse_moq(self._resolve_text(moq_el) or "")
            data["ventas"] = limpiar_cantidad(self._resolve_text(self._first_match(card, self.SOLD_COUNT))) 
            
            # --- FASE 3: Proveedor y Calidad ---
            proveedor_el = self._first_match(card, self.SUPPLIER_NAME)
            data["proveedor"] = self._resolve_text(proveedor_el)
            data["proveedor_verificado"] = bool(self._first_match(card, self.VERIFIED_BADGE))
            
            year_ctry_el = self._first_match(card, self.SUPPLIER_YEAR_COUNTRY)
            data["proveedor_anios"], data["proveedor_pais"] = parse_years_country(year_ctry_el) if year_ctry_el else (None, None)
            
            rating_el = self._first_match(card, self.RATING)
            rating_text = self._resolve_text(rating_el)
            data["rating_score"], data["rating_count"] = parse_rating(rating_text or "")
            
            # --- FASE 4: Metadatos Avanzados ---
            
            m = _IS_P4P_RE.search(aplus_data); data["is_p4p"] = (m and m.group(1) == 'true')
            m = _RLT_RANK_RE.search(aplus_data); data["rlt_rank"] = int(m.group(1)) if m and m.group(1).isdigit() else None
            data["es_anuncio"] = data.get("is_p4p", False) or bool(self._first_match(card, self.AD_BADGE))
            
            return data
        except (NoSuchElementException, StaleElementReferenceException): 
            # logging.warning("[DIAGNÓSTICO] Tarjeta omitida por StaleElement o elemento no encontrado.")
            return None
        except Exception as e:
            logging.error(f"[DIAGNÓSTICO] Error FATAL en extracción: {e}")
            return None


    @staticmethod
    def _is_blocked(driver):
        url = (getattr(driver, "current_url", "") or "").lower()
        if any(p in url for p in ["punish", "robot check", "verify you are human"]): return True
        return False

    def parse(self, producto: str, paginas: int = 4):
        try:
            resultados: List[Dict] = []
            
            COLUMN_ORDER = [
                "product_id", "titulo", "precio", "moneda", "precio_original", 
                "ventas", "moq", "proveedor_verificado", "proveedor_anios", "proveedor_pais",
                "rating_score", "es_anuncio", "is_p4p", "rlt_rank", "link", 
                "fecha_scraping"
            ]

            for page in range(1, paginas + 1):
                q = quote_plus(producto)
                url = f"https://www.alibaba.com/trade/search?SearchText={q}&page={page}"
                logging.info(f"Cargando Alibaba: Página {page} -> {url}")

                cargada = False
                for intento in range(3):
                    try:
                        self.driver.get(url)
                        self._accept_banners(5)
                        WebDriverWait(self.driver, 15).until(
                            EC.visibility_of_any_elements_located((By.CSS_SELECTOR, ", ".join(self.CARD_CONTAINERS)))
                        )
                        self._human_scroll_until_growth(max_scrolls=16, pause=1.0)
                        cargada = True
                        break
                    except (TimeoutException, WebDriverException) as e:
                        logging.warning(f"Reintento Alibaba p{page} ({intento + 1}): {e}")
                        time.sleep(1.0)

                if not cargada:
                    logging.error(f"Omitiendo página {page} por fallos de carga.")
                    continue

                if self._is_blocked(self.driver):
                    logging.warning(f"Posible bloqueo/antibot detectado en Alibaba (página {page}).")

                bloques = self._find_all_any(self.CARD_CONTAINERS, timeout=8)
                logging.info(f"Página {page}: {len(bloques)} productos (candidatos via Selenium)")

                count_page = 0
                for card in bloques:
                    data = self._extract_card(card)
                    if not data or (not data.get('link') and data.get('titulo') == 'Sin título'): continue
                    
                    final_data = {col: data.get(col) for col in COLUMN_ORDER}
                    final_data.update({
                        "pagina": page,
                        "plataforma": "Alibaba",
                        "fecha_scraping": datetime.now().strftime("%Y-%m-%d"),
                        "producto_busqueda": producto 
                    })
                    
                    resultados.append(final_data)
                    count_page += 1

                logging.info(f"Página {page}: {count_page} productos válidos (Selenium)")

            return resultados
        finally:
            pass 

# ------------------- BLOQUE DE EJECUCIÓN DEL NOTEBOOK (ITINERARIO) -------------------

if __name__ == '__main__':
    # 🚨 LISTA DE PRODUCTOS A BUSCAR
    PRODUCTOS_BUSQUEDA = [
        "CAMISA MANGA LARGA", "CAMISA MANGA CORTA", "ACCESORIOS", "CAMISA", "POLOS MANGA CORTA HOMBRE SPORT", 
        "PANTALON", "BERMUDA", "CORBATA", "SACO", "TERNO", "CALZADO", "CHALECO", "CHOMPA", 
        "PIJAMA", "CAMISA MANGA LARGA SPORT", "CORREA", "PAÑUELO", "MEDIAS DEPORTIVAS", "CASACA", 
        "POLO MANGA CORTA", "BIKINI", "CAMISETA", "BOXER", "CALCETIN", "PULLOVER", "POLERA", 
        "ZAPATILLA", "CORREA TEXTIL MODA", "BUZO", "CARDIGAN", "ABRIGO", "DENIM", 
        "POLOS MC NIÑO SPORT", "PANTALONES SPORT NIÑO", "POLERA SPORT HOMBRE", "HOODIES", 
        "RELOJ", "ZAPATILLAS", "POLO SPORT MANGA LARGA", "ROPA DE BAÑO", "ROPA DE BAÑO"
    ]
    
    PAGINAS_A_SCRAPEAR = 10 # Se respeta la solicitud de 10 páginas
    OUTPUT_EXCEL_FILE = "productos_alibaba_MAESTRO.xlsx" # Archivo único de salida

    todos_los_resultados = []
    
    logging.info("Iniciando Scraper de Alibaba para el itinerario completo...")
    
    try:
        scraper = AlibabaScraper() 

        for producto in PRODUCTOS_BUSQUEDA:
            logging.info(f"\n=======================================================")
            logging.info(f"== INICIANDO PRODUCTO: {producto} | {PAGINAS_A_SCRAPEAR} PÁGINAS ==")
            logging.info(f"========================================================")
            
            # Ejecuta el parseo para el producto actual
            resultados_producto = scraper.parse(producto, PAGINAS_A_SCRAPEAR)
            
            if resultados_producto:
                todos_los_resultados.extend(resultados_producto)

        # 3. Convierte a DataFrame y guarda después de todos los productos
        if todos_los_resultados:
            df_resultados = pd.DataFrame(todos_los_resultados)
            
            COLUMN_ORDER_FINAL = [
                "producto_busqueda", "product_id", "titulo", "precio", "moneda", "precio_original", 
                "ventas", "moq", "proveedor_verificado", "proveedor_anios", "proveedor_pais",
                "rating_score", "es_anuncio", "is_p4p", "rlt_rank", "link", 
                "fecha_scraping", "pagina", "plataforma"
            ]
            
            df_resultados = df_resultados[[col for col in COLUMN_ORDER_FINAL if col in df_resultados.columns]]
            
            # Exportación a Excel (Requiere openpyxl)
            df_resultados.to_excel(OUTPUT_EXCEL_FILE, index=False)
            
            logging.info(f"\n========================================================")
            logging.info(f"✅ EXTRACCIÓN MAESTRA COMPLETADA: {len(df_resultados)} productos.")
            logging.info(f"Resultados guardados en {OUTPUT_EXCEL_FILE}")
            logging.info(f"========================================================")
            
            print("\n--- Vista Previa de los Resultados Maestros ---")
            print(df_resultados.head().to_markdown(index=False))
            
        else:
            logging.warning("No se encontraron resultados válidos en todo el itinerario.")

    except Exception as e:
        logging.error(f"Ocurrió un error inesperado durante el scraping: {e}")
    finally:
        # 4. Cierra el driver
        if scraper:
            scraper.close()
            logging.info("Navegador Selenium cerrado.")

2025-12-09 12:05:45,075 - INFO - ====== WebDriver manager ======



--- 🌐 Paso 1: Scraping de Búsqueda en ALIBABA para 'camisa' ---


2025-12-09 12:05:47,263 - INFO - Get LATEST chromedriver version for google-chrome
2025-12-09 12:05:47,277 - INFO - Get LATEST chromedriver version for google-chrome
2025-12-09 12:05:47,293 - INFO - Driver [C:\Users\alejo\.wdm\drivers\chromedriver\win64\142.0.7444.175\chromedriver-win32/chromedriver.exe] found in cache
2025-12-09 12:05:48,305 - INFO - Cargando Alibaba: Página 1
2025-12-09 12:06:51,432 - INFO - Página 1: 47 productos válidos.


✅ Búsqueda completada: 47 productos encontrados.


2025-12-09 12:06:53,547 - INFO - WebDriver cerrado.
2025-12-09 12:06:53,554 - INFO - ====== WebDriver manager ======



--- 🔎 Paso 2: Scraping de Detalles para 3 Enlaces ---


2025-12-09 12:06:55,889 - INFO - Get LATEST chromedriver version for google-chrome
2025-12-09 12:06:55,903 - INFO - Get LATEST chromedriver version for google-chrome
2025-12-09 12:06:55,914 - INFO - Driver [C:\Users\alejo\.wdm\drivers\chromedriver\win64\142.0.7444.175\chromedriver-win32/chromedriver.exe] found in cache
2025-12-09 12:06:56,995 - INFO - Cargando detalles: https://www.alibaba.com/product-detail/Oversized-Embroidered-Striped-Shirt-Womens-Loose_1601588258560.html
2025-12-09 12:07:10,103 - INFO - Cargando detalles: https://www.alibaba.com/product-detail/Long-sleeved-Shirt-230g-Cotton-Loose_1601638704623.html
2025-12-09 12:07:23,473 - INFO - Cargando detalles: https://www.alibaba.com/product-detail/Unisex-Custom-Men-s-Long-Sleeve_1601463834391.html
2025-12-09 12:07:38,748 - INFO - WebDriver cerrado.



--- ✅ ÉXITO Y EXPORTACIÓN ---
Datos combinados (Búsqueda + Detalles) exportados a: alibaba_analisis_camisa_20251209_1207.csv


,plataforma,titulo,precio,moneda,proveedor,ingresos_anuales_usd,peso_bruto_kg,precios_por_niveles
0,Alibaba,Oversized Embroidered Striped Shirt Womens Loo...,8.99,$,"Dongguan Baijing Garments Co., Ltd.",None,None,"[{'cantidad_rango': '1 - 19 pieces', 'precio_u..."
1,Alibaba,Long-sleeved Shirt 230g Cotton Loose Round Nec...,2.97,$,Suning County Benlei Trading Store,None,None,"[{'cantidad_rango': '20 - 199 pieces', 'precio..."
2,Alibaba,Unisex Custom Men's Long Sleeve Casual Shirt E...,15.96,$,"Huizhou Honglai Clothing Co., Ltd.",None,None,"[{'cantidad_rango': '50 - 4950 pieces', 'preci..."
3,Alibaba,Wholesale Custom 100% Polyester Casual Blank R...,5.00,$,RAIEBA ENTERPRISES,NaN,NaN,NaN
4,Alibaba,Freedom Charlie Kirk Unisex T-Shirt 100% Cotto...,4.00,$,HADEED ENTERPRISES,NaN,NaN,NaN
